
Semestral Work Assignment (4IZ172) – Clustering
# General information
One of two possible choices
The final form is a Jupyter Notebook
Recommended languages: Python, R
Follow the outline sketched below


In [43]:
import pandas as pd
import chardet


# Dataset
3 choices -- 3 datasets will be prepared in the following days: diploma theses, aerospace enterprises, scientific results

The section should contain:

Description of the dataset
Basic statistics / reports about the selected dataset, incl. linguistic and textual features


In [99]:
with open('data/RawDatasetFISbachelor.csv', 'rb') as f:
    rawdata = f.read()
result = chardet.detect(rawdata)
encoding = result['encoding']
print(f"Detected encoding: {encoding}")

Detected encoding: Windows-1254


In [132]:

df_aero = pd.read_csv("data/FR-aerospace-translate.csv")

df_fis = pd.read_csv("data/RawDatasetFISbachelor.csv",encoding="UTF-8-SIG", lineterminator='\r',sep=";")
df_fis = df_fis.replace(r'\n',' ', regex=True) 

df_utia = pd.read_csv("data/UTIAresults.csv")


# Preprocessing
The section should contain:

Description of preprocessing steps involved
Report on the preprocessing results



# Clustering
This section should contain:

List of clustering methods considered
Estimation of the number of clusters (methods, results, discussion)
Comparison of (the results of) clustering methods on the selected dataset



# Cluster Descriptions
The clusters (at least important part of them) should be described.

This section should contain:

Description of the selected approach
Examples of the descriptions obtained



# # Visualization
This section should contain:

At least one visualization related to clustering



# Analytical Report
The entire work should be ended by the final analytical report of findings on the selected dataset. It should point out your main results, their justifications, hypotheses, conclusions, outliers, ...